In [1]:
import sys
sys.path.append('/opt/landmark/experiments/deep-image-retrieval/')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

In [5]:
import argparse
import os
import time
import pickle
import pdb

import numpy as np

import torch
from torch.nn import Parameter
from torch.utils.model_zoo import load_url
from torchvision import transforms

from cirtorch.networks.imageretrievalnet import init_network, extract_vectors
from cirtorch.datasets.testdataset import configdataset
from cirtorch.utils.download import download_train, download_test
from cirtorch.utils.evaluate import compute_map_and_print
from cirtorch.utils.general import get_data_root, htime

PRETRAINED = {
    'rSfM120k-tl-resnet50-gem-w': 'http://cmp.felk.cvut.cz/cnnimageretrieval/data/networks/retrieval-SfM-120k/rSfM120k-tl-resnet50-gem-w-97bf910.pth',
    'rSfM120k-tl-resnet101-gem-w': 'http://cmp.felk.cvut.cz/cnnimageretrieval/data/networks/retrieval-SfM-120k/rSfM120k-tl-resnet101-gem-w-a155e54.pth',
    'rSfM120k-tl-resnet152-gem-w': 'http://cmp.felk.cvut.cz/cnnimageretrieval/data/networks/retrieval-SfM-120k/rSfM120k-tl-resnet152-gem-w-f39cada.pth',
    'gl18-tl-resnet50-gem-w': 'http://cmp.felk.cvut.cz/cnnimageretrieval/data/networks/gl18/gl18-tl-resnet50-gem-w-83fdc30.pth',
    'gl18-tl-resnet101-gem-w': 'http://cmp.felk.cvut.cz/cnnimageretrieval/data/networks/gl18/gl18-tl-resnet101-gem-w-a4d43db.pth',
    'gl18-tl-resnet152-gem-w': 'http://cmp.felk.cvut.cz/cnnimageretrieval/data/networks/gl18/gl18-tl-resnet152-gem-w-21278d5.pth',
}

# datasets_names = ['oxford5k', 'paris6k', 'roxford5k', 'rparis6k']
datasets_names = ['roxford5k', 'rparis6k']

state = load_url(PRETRAINED['gl18-tl-resnet101-gem-w'], model_dir=os.path.join(
    get_data_root(), 'networks'))

net_params = {}
net_params['architecture'] = state['meta']['architecture']
net_params['pooling'] = state['meta']['pooling']
net_params['local_whitening'] = state['meta'].get('local_whitening', False)
net_params['regional'] = state['meta'].get('regional', False)
net_params['whitening'] = state['meta'].get('whitening', False)
net_params['mean'] = state['meta']['mean']
net_params['std'] = state['meta']['std']
net_params['pretrained'] = False
# network initialization
net = init_network(net_params)
net.pool.p = Parameter(torch.ones(1) * 3.0)
net.load_state_dict(state['state_dict'])

print(">>>> loaded network: ")
print(net.meta_repr())

# moving network to gpu and eval mode
net.cuda()
net.eval()
model = net
model.extract_feat = lambda x: model(x).t()
model.DIVIDABLE_BY = 32

>>>> loaded network: 
  (meta): dict( 
     architecture: resnet101
     local_whitening: False
     pooling: gem
     regional: False
     whitening: True
     outputdim: 2048
     mean: [0.485, 0.456, 0.406]
     std: [0.229, 0.224, 0.225]
  )



ImageRetrievalNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d(64,

In [19]:
from src.eval_retrieval import eval_datasets
datasets = ('roxford5k', 'rparis6k')
results = eval_datasets(model, datasets=datasets, ms=True, tta_gem_p=1.0)

roxford5k: 100%|███████| 70/70 [00:04<00:00, 14.22it/s]


>> roxford5k: mAP E: 84.37, M: 67.22, H: 43.61
>> roxford5k: mP@k[1, 5, 10] E: [94.12 90.29 86.99], M: [94.29 90.   84.86], H: [87.14 69.14 59.43]


rparis6k: 100%|████████| 70/70 [00:04<00:00, 14.83it/s]


>> rparis6k: mAP E: 92.94, M: 80.62, H: 61.39
>> rparis6k: mP@k[1, 5, 10] E: [98.57 96.57 95.29], M: [100.    98.    96.86], H: [97.14 93.43 91.14]


In [ ]:
from src import utils, data_utils, metrics
from tqdm import tqdm
import torch
# import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from pathlib import Path
import h5py

ROOT = '/opt/landmark/'
params = {
    'ex_name': 'eval_triplet',
    'test_batch_size': 16,
    'fc_dim': 2048,
}
splits = ['test19', 'index19']
scale = 'L2'
ms = True
n_blocks = 1
block_id = 0

train_transform, eval_transform = data_utils.build_transforms()

data_loaders = data_utils.make_predict_loaders(params,
                                               eval_transform=eval_transform,
                                               scale=scale,
                                               splits=splits,
                                               num_workers=8,
                                               n_blocks=n_blocks,
                                               block_id=block_id)

model_path = ROOT + '../data/networks/gl18-tl-resnet101-gem-w-a4d43db.pth'
exp_path = ROOT + f'experiments/{params["ex_name"]}/'
if not Path(exp_path).exists():
    Path(exp_path).mkdir(parents=True)

file_suffix = model_path.split('/')[-1].replace('.pth', '')
file_suffix = scale + '_' + file_suffix
file_suffix = 'ms_' + file_suffix if ms else file_suffix

min_size = 128
scales = [0.75, 1.0, 1.25] if ms else [1.0]

for split in splits:
    ids, feats = [], []
    for i, (img_id, x) in tqdm(enumerate(data_loaders[split]),
                               total=len(data_loaders[split]),
                               miniters=None, ncols=55):

        batch_size, _, h, w = x.shape
        feat_blend = np.zeros((batch_size, params['fc_dim']), dtype=np.float32)

        with torch.no_grad():
            x = x.to('cuda')

            for s in scales:
                th = max(min_size, int(h * s // model.DIVIDABLE_BY * model.DIVIDABLE_BY))
                tw = max(min_size, int(w * s // model.DIVIDABLE_BY * model.DIVIDABLE_BY))  # round off

                scaled_x = F.interpolate(x, size=(th, tw), mode='bilinear', align_corners=True)
                feat = model.extract_feat(scaled_x)
                feat = feat.cpu().numpy()
                feat_blend += feat

        feats.append(feat_blend)
        ids.extend(img_id)

    feats = np.concatenate(feats) / len(scales)

    output_path = Path(f'{exp_path}feats_{split}_{file_suffix}')
    output_path.mkdir(parents=True, exist_ok=True)
    with h5py.File(output_path / f'block{block_id}.h5', 'a') as f:
        f.create_dataset('ids', data=np.array(ids, dtype=f'S{len(ids[0])}'))
        f.create_dataset('feats', data=feats)


 50%|██████▉       | 3655/7351 [43:28<47:14,  1.30it/s]

In [1]:
import sys
sys.path.append('/opt/landmark/experiments/deep-image-retrieval/')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5'

from dirtorch.utils import common
import dirtorch.nets as nets
from src.eval_retrieval import eval_datasets

def load_model(path, iscuda):
    checkpoint = common.load_checkpoint(path, iscuda)
    net = nets.create_model(pretrained="", **checkpoint['model_options'])
    net = common.switch_model_to_cuda(net, iscuda, checkpoint)
    net.load_state_dict(checkpoint['state_dict'])
    net.preprocess = checkpoint.get('preprocess', net.preprocess)
    if 'pca' in checkpoint:
        net.pca = checkpoint.get('pca')
    return net

model = load_model('../experiments/Resnet101-AP-GeM-LM18.pt', True)
model = model.module.eval()

In [9]:
datasets = ('roxford5k', 'rparis6k')
results = eval_datasets(model, datasets=datasets, ms=True, tta_gem_p=1.0)

roxford5k: 100%|███████| 70/70 [00:04<00:00, 14.47it/s]


>> roxford5k: mAP E: 82.03, M: 65.77, H: 40.87
>> roxford5k: mP@k[1, 5, 10] E: [92.65 87.65 86.59], M: [92.86 87.71 82.43], H: [81.43 66.29 54.14]


rparis6k: 100%|████████| 70/70 [00:04<00:00, 15.04it/s]


>> rparis6k: mAP E: 92.52, M: 81.2, H: 61.62
>> rparis6k: mP@k[1, 5, 10] E: [98.57 96.86 95.67], M: [100.    98.    97.57], H: [97.14 92.57 91.  ]


In [1]:
import sys
sys.path.append('/opt/landmark/experiments/deep-image-retrieval/')
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5'

from dirtorch.utils import common
import dirtorch.nets as nets
from src.eval_retrieval import eval_datasets

def load_model(path, iscuda):
    checkpoint = common.load_checkpoint(path, iscuda)
    net = nets.create_model(pretrained="", **checkpoint['model_options'])
    net = common.switch_model_to_cuda(net, iscuda, checkpoint)
    net.load_state_dict(checkpoint['state_dict'])
    net.preprocess = checkpoint.get('preprocess', net.preprocess)
    if 'pca' in checkpoint:
        net.pca = checkpoint.get('pca')
    return net

In [ ]:
from src import utils, data_utils, metrics
from tqdm import tqdm
import torch
# import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from pathlib import Path
import h5py

ROOT = '/opt/landmark/'
params = {
    'ex_name': 'eval_ap',
    'test_batch_size': 16,
    'fc_dim': 2048,
}
splits = ['test19', 'index19']
scale = 'L2'
ms = True
n_blocks = 1
block_id = 0

train_transform, eval_transform = data_utils.build_transforms()

data_loaders = data_utils.make_predict_loaders(params,
                                               eval_transform=eval_transform,
                                               scale=scale,
                                               splits=splits,
                                               num_workers=8,
                                               n_blocks=n_blocks,
                                               block_id=block_id)

model_path = ROOT + 'experiments/Resnet101-AP-GeM-LM18.pt'
exp_path = ROOT + f'experiments/{params["ex_name"]}/'
if not Path(exp_path).exists():
    Path(exp_path).mkdir(parents=True)

model = load_model(model_path, True)
model = model.module.eval()

file_suffix = model_path.split('/')[-1].replace('.pth', '')
file_suffix = scale + '_' + file_suffix
file_suffix = 'ms_' + file_suffix if ms else file_suffix

min_size = 128
scales = [0.75, 1.0, 1.25] if ms else [1.0]

for split in splits:
    ids, feats = [], []
    for i, (img_id, x) in tqdm(enumerate(data_loaders[split]),
                               total=len(data_loaders[split]),
                               miniters=None, ncols=55):

        batch_size, _, h, w = x.shape
        feat_blend = np.zeros((batch_size, params['fc_dim']), dtype=np.float32)

        with torch.no_grad():
            x = x.to('cuda')

            for s in scales:
                th = max(min_size, int(h * s // model.DIVIDABLE_BY * model.DIVIDABLE_BY))
                tw = max(min_size, int(w * s // model.DIVIDABLE_BY * model.DIVIDABLE_BY))  # round off

                scaled_x = F.interpolate(x, size=(th, tw), mode='bilinear', align_corners=True)
                feat = model.extract_feat(scaled_x)
                feat = feat.cpu().numpy()
                feat_blend += feat

        feats.append(feat_blend)
        ids.extend(img_id)

    feats = np.concatenate(feats) / len(scales)

    output_path = Path(f'{exp_path}feats_{split}_{file_suffix}')
    output_path.mkdir(parents=True, exist_ok=True)
    with h5py.File(output_path / f'block{block_id}.h5', 'a') as f:
        f.create_dataset('ids', data=np.array(ids, dtype=f'S{len(ids[0])}'))
        f.create_dataset('feats', data=feats)


In [7]:
import torch
x = torch.rand((1, 3, 256, 256)).cuda()
y = model(x.cuda())

y.shape

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
from collections import Counter
import faiss
import tqdm
import numpy as np
import pandas as pd
import subprocess
from src import utils
from src.reranking import Diffusion, explore_exploit
# from experiments.submit_retrieval import predict_landmark_id

ROOT = '../'

setting = 'AP_Loss'
index_dirs = ['../experiments/eval_ap/feats_index19_ms_L2_Resnet101-AP-GeM-LM18.pt/']
test_dirs = ['../experiments/eval_ap/feats_test19_ms_L2_Resnet101-AP-GeM-LM18.pt/']
# index_dirs = ['../experiments/eval_triplet/feats_index19_ms_L2_gl18-tl-resnet101-gem-w-a4d43db/']
# test_dirs = ['../experiments/eval_triplet/feats_test19_ms_L2_gl18-tl-resnet101-gem-w-a4d43db/']

weights = [1.0]

ids_index, feats_index = utils.prepare_ids_and_feats(index_dirs, weights, normalize=True)
ids_test, feats_test = utils.prepare_ids_and_feats(test_dirs, weights, normalize=True)

co = faiss.GpuMultipleClonerOptions()
co.shard = True

vres = []
for _ in range(2):
    res = faiss.StandardGpuResources()
    vres.append(res)

print('build index...')
cpu_index = faiss.IndexFlatIP(feats_index.shape[1])
gpu_index = faiss.index_cpu_to_gpu_multiple_py(vres, cpu_index, co)
gpu_index.add(feats_index)
sims, topk_idx = gpu_index.search(x=feats_test, k=100)
print('query search done.')

subm = pd.DataFrame(ids_test, columns=['id'])
subm['images'] = np.apply_along_axis(' '.join, axis=1, arr=ids_index[topk_idx])

output_name = f'../output/{setting}.csv.gz'
subm[['id', 'images']].to_csv(output_name, compression='gzip', index=False)
print('saved to ' + output_name)

cmd = f'kaggle c submit -c landmark-retrieval-2019 -f {output_name} -m "" '
print(cmd)
subprocess.run(cmd, shell=True)

build index...
query search done.
saved to ../output/AP_Loss.csv.gz
